In [1]:
from __future__ import print_function
import time
import json

import requests
import pandas as pd

import tkinter as tk
from tkinter import filedialog

from six import StringIO

In [2]:
# Code related to Kegg compound records

def get_kegg_section(k_record, sname):
    """Get the section with name _sname_ from a kegg compound record.
    
       Returns a str, possibly empty."""
    
    in_section = False
    section = []
    
    for line in k_record.splitlines():
        if line.startswith(sname):
            in_section = True
            section.append(line)
        elif in_section and line.startswith(' '):
            section.append(line)
        elif in_section and not line.startswith(' '):
            break

    sectionlines = [line[12:] for line in section]
    return '\n'.join(sectionlines)


def get_kegg_records(db_fname):
    """Load Kegg compound records from a local text DB."""
    kegg = {}
    db = open(db_fname)
    curr_c = None
    curr_txt = None
    for line in db:
        if len(line.strip()) == 0:
            continue
        if line.startswith('---- Compound:'):
            if curr_c is not None:
                whole_rec = ''.join(curr_txt)
                kegg[curr_c] = whole_rec
            curr_c = line.split(':')[1].strip()
            curr_txt = []
        elif curr_c is not None:
            curr_txt.append(line)
    whole_rec = ''.join(curr_txt)
    kegg[curr_c] = whole_rec
    db.close()
    return kegg


def get_dblinks_brite_sections(k_record):
    """Get sections DBLINKS and BRITE from a kegg compound record.
    
       Returns a dict."""
    
    dblinks = get_kegg_section(k_record, 'DBLINKS')
    brite = get_kegg_section(k_record, 'BRITE')
    return {'DBLINKS':dblinks, 'BRITE':brite}

In [3]:
#def get_kegg_record(c, fstore=None):
    #krecord = requests.get('http://rest.kegg.jp/get/' + c).text
    #if fstore is not None:
        #fstore.write('\n---- Compound: {}\n{}'.format(c, krecord))
    #return krecord

def get_kegg_record(c_id, localdict=None):
    if localdict is not None:
        return localdict[c_id]
    return requests.get('http://rest.kegg.jp/get/' + c_id).text

def get_lipidmaps_record(lm_id):
    return requests.get('http://www.lipidmaps.org/rest/compound/lm_id/' + lm_id + '/all/json').text

In [4]:
# Helper. No used in example code (file names are hard coded).
def get_filename_using_tk():
    """Choose a filename using Tk"""
    root = tk.Tk()
    root.withdraw()
    fname = filedialog.askopenfilename(filetypes = [("TSV","*.tsv")])
    print ('Selected file {}'.format(fname))
    return fname

In [5]:
def readMassTRIX(fname):
    """Reads a MassTRIX file into a Pandas DataFrame object.
       
       On the process, the last line is moved to the beginning and becomes the header."""
    
    # store lines in a list
    with open(fname) as f:
        lines = [line.strip() for line in f]
    
    # move the last line to the beginning
    moved_list = [lines[-1]]
    moved_list.extend(lines[:-1]) # last line is not included

    # create a Pandas DataFrame, reading from the list of strings in memory
    mem_string = StringIO('\n'.join(moved_list))
    df = pd.read_table(mem_string)
    return df

In [6]:
class Progress(object):
    """A progress reporter of % done for a maximum (int) of self.total"""
    def __init__(self, n_total=1):
        self.total = n_total
        self.count = 0
    def tick(self):
        self.count +=1
        print(str(round((1-((self.total-self.count)/self.total))*100)) + "% done")

In [7]:
print ('Starting...\n')
start_time = time.time()

Starting...



In [8]:
kegg_db = get_kegg_records('kegg_db.txt')

In [9]:
testfile_name = 'example_data/MassTRIX_output.tsv'

In [10]:
df = readMassTRIX(testfile_name)

# Retira as colunas 9 a 20, que não são necessárias
# Exprimentei df.drop(df.columns[['nome', 'nome 2', ..., 'nome n']], axis=1, inplace=True) mas não resulta
df.drop(df.columns[[9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]], axis=1, inplace=True)

# check result
df.info() # assert that there are 15 entries and 24 - 12 = 12 columns
print(df.head(2))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 12 columns):
raw_mass                      15 non-null float64
peak_height                   15 non-null float64
corrected_mass                15 non-null float64
npossible                     15 non-null int64
KEGG_mass                     15 non-null object
ppm                           15 non-null object
KEGG_cid                      15 non-null object
KEGG_formula                  15 non-null object
KEGG_name                     15 non-null object
KEGG Pathways                 15 non-null object
KEGG Pathways descriptions    15 non-null object
Compound in Organism(X)       15 non-null object
dtypes: float64(3), int64(1), object(8)
memory usage: 1.5+ KB
    raw_mass  peak_height  corrected_mass  npossible  \
0  133.08577          0.0      132.078494         26   
1  149.01145          0.0      110.048291          3   

                                           KEGG_mass  \
0  133.085920710401#

In [11]:
def classes_from_kegg(c):
    krecord = get_kegg_record(c, localdict = kegg_db)
    
    brite = get_kegg_section(krecord, 'BRITE')
    dblinks = get_kegg_section(krecord, 'DBLINKS')
    
    if len(brite) == 0:
        if 'LIPIDMAPS:' in dblinks:
            for line in dblinks.splitlines():
                if 'LIPIDMAPS:' in line:
                    lm_id = line.split(':')[1].strip()
                    a = classes_from_lipidmaps(lm_id)
                    return a
        else:
            print ('No BRITE, no LIPIDMAPS')
            return None
    else:
        classes = [[], [], [], []]
        
        k = krecord.split('BRIT', 1)[1]
        k = k.split('DBLINKS', 1)[0]
        k = k.splitlines()
        l = []
        for x in k:
            if x.startswith('E       '):
                l.append('            ' + x[len('E       '):])
            else:
                l.append(x)
        l = "\n".join(l)
        
        l = l.split('           ')[1:]
        p = []
        for x in l:
            if not x.startswith('  '):
                x = ('??'+x)
                p.append(x)
            else:
                p.append(x)
        p = "".join(p)
        p = p.split('??')
        p = "\n".join(p)
        p = p.splitlines()
        for x in p:
            if x.startswith(' ') and not x.startswith('  '):
                classes[0].append(x[1:])
            elif x.startswith('  ') and not x.startswith('   '):
                classes[1].append(x[2:])
            elif x.startswith('   ') and not x.startswith('    '):
                classes[2].append(x[3:])
            elif x.startswith('    ') and not x.startswith('     '):
                classes[3].append(x[4:])
        
        classes = ['#'.join(c) for c in classes]
        print ('from BRITE:')
        print(tuple(classes))
        return tuple(classes)

In [12]:
# Anota os compostos do LipidMaps
def classes_from_lipidmaps(lm_id):
    print('LIPIDMAPS id: {}'.format(lm_id))
    f = requests.get('http://www.lipidmaps.org/rest/compound/lm_id/' + lm_id + '/all/json').text
    s = json.loads(f)
    if f == '[]':
        return 'null', 'null', 'null', 'null'
    mm = 'Lipids [LM]'
    cc = s['core'] if s['core'] is not None else 'null'
    ss = s['main_class'] if s['main_class'] is not None else 'null'
    tt = s['sub_class'] if s['sub_class'] is not None else 'null'
    a = (mm, cc, ss, tt)
    print(a)
    return a 

In [13]:
def annotate_classes(x):
    """Create Pandas Series with compound class annotations.
       
       Information is fetched from Kegg or LIPIDMAPS, according to first
       letter of compound Id."""
    
    classes = [[], [], [], []]
    for y in x.split('#'):
        print('\n---- compound: {}'.format(y), end=" ")
        if y.startswith('C'):
            k_classes = classes_from_kegg(y)
            if k_classes is not None :
                for c, k in zip(classes, k_classes):
                    c.append(k)
        elif y.startswith('L'):
            lipid_classes = classes_from_lipidmaps(y)
            for c, k in zip(classes, lipid_classes):
                c.append(k)
        else:
            continue
    progress.tick()
    
    classes = ['#'.join(set(c)) for c in classes]    
    return pd.Series(classes, index=['Major Class', 'Class', 'Secondary Class', 'Tertiary Class'])

In [14]:
def is_in_plants(c):
    """Annotates the presence in plants, by using information in KNApSAcK DB."""
    m = get_kegg_record(c, localdict = kegg_db)
    #m = requests.get('http://rest.kegg.jp/get/' + c).text
    dblinks = get_kegg_section(m, 'DBLINKS')
    if 'KNApSAcK' in dblinks:
        for line in dblinks.splitlines():
            if 'KNApSAcK:' in line:
                ks_id = line.split(':')[1].strip()
                r = requests.post('http://kanaya.naist.jp/knapsack_jsp/information.jsp?sname=C_ID&word=' + ks_id)
                result = 'Plantae' if 'Plantae' in r.text else ''
                return result
    return ''

def knapsack_plants(x):
    knapsack=[]
    for y in x.split('#'):
        if y.startswith('C'):
            knapsack.append(is_in_plants(y))
    # clear_output()
    progress.tick()
    return "".join(set(knapsack))

In [15]:
# Load ID translation tables as dicts
# Use local files, (fetched by fetch_dbs.py)
def get_trans_id_table(fname):
    d = {}
    with open(fname) as f:
        # hmdb:HMDB00002 \t cpd:C00986 \t equivalent
        for line in f:
            if len(line) == 0:
                continue
            foreign, cpd, equiv = line.split('\t')
            foreign = foreign.split(':')[1].strip()
            cpd = cpd.split(':')[1].strip()
            d[foreign] = cpd
    return d

hmdb2kegg_dict = get_trans_id_table('trans_hmdb2kegg.txt')
lipidmaps2kegg_dict = get_trans_id_table('trans_lipidmaps2kegg.txt')

kegg2lipidmaps_dict = {}
for k, v in lipidmaps2kegg_dict.items():
    kegg2lipidmaps_dict[v] = k

In [16]:
# Criar coluna de conversão HMDB to KEGG
def translate_id(x, tdict):
    convert=[]
    for y in x.split('#'):
        if y in tdict:
            convert.append(tdict[y])
        else:
            convert.append(y)
    return "#".join(convert)

In [17]:
# Criar coluna de conversão HMDB to KEGG
def hmdb2kegg(x):
    convert=[]
    for y in x.split('#'):
        if y in hmdb2kegg_dict:
            convert.append(hmdb2kegg_dict[y])
        else:
            convert.append(y)
    return "#".join(convert)

In [18]:
# Criar coluna de conversão KEGG para Lipid Maps
def kegg2lipidmaps(x):
    convert=[]
    for y in x.split('#'):
        if y in kegg2lipidmaps_dict:
            convert.append(kegg2lipidmaps_dict[y])
        else:
            convert.append(y)
    return "#".join(convert)

In [19]:
# Criar coluna de convertidos KEGG para Lipid Maps
def lipidmaps2kegg(x):
    convert=[]
    for y in x.split('#'):
        if y in lipidmaps2kegg_dict:
            convert.append(lipidmaps2kegg_dict[y])
        else:
            convert.append(y)
    return "#".join(convert)

In [20]:
# Object to report progress of annotations. It is a two-pass job.
progress = Progress(len(df['raw_mass']) * 2)

In [21]:
# Converte os códigos HMDB para KEGG, se possivél, através do dicionário "hmdb2kegg"
#df['HMDB_2_KEGG'] = df['KEGG_cid'].apply(translate_id, args=(dic,))
df['HMDB_2_KEGG'] = df['KEGG_cid'].apply(hmdb2kegg)

# Converte os códigos KEGG para Lipid Maps, se possível, através do dicionário "kegg2lipidmaps"
df['KEGG_2_LipidMaps'] = df['HMDB_2_KEGG'].apply(kegg2lipidmaps)

# Converte os códigos Lipid Maps para KEGG, se possível, através do dicionário "lipidmaps2kegg"
df['LipidMaps_2_KEGG'] = df['HMDB_2_KEGG'].apply(lipidmaps2kegg)

In [22]:
elapsed_time = time.time() - start_time
m, s = divmod(elapsed_time, 60)
print ("Created translation columns in " + "%02dm%02ds" % (m, s))

Created translation columns in 00m02s


In [23]:
# Create 4 new columns with class hierarchy.
df = pd.concat([df, df['KEGG_2_LipidMaps'].apply(annotate_classes)], axis=1)


---- compound: C03264 LIPIDMAPS id: LMFA01050402
('Lipids [LM]', 'Fatty Acyls [FA]', 'Fatty Acids and Conjugates [FA01]', 'Hydroxy fatty acids [FA0105]')

---- compound: C03499 No BRITE, no LIPIDMAPS

---- compound: C06103 from BRITE:
('Lipids [BR:br08002]', 'FA  Fatty acyls', 'FA01 Fatty Acids and Conjugates', 'FA0105 Hydroxy fatty acids')

---- compound: C07834 from BRITE:
('Anatomical Therapeutic Chemical (ATC) classification [BR:br08303]', 'N NERVOUS SYSTEM', 'N05 PSYCHOLEPTICS', 'N05C HYPNOTICS AND SEDATIVES')

---- compound: HMDB00317 
---- compound: HMDB00409 
---- compound: HMDB00525 
---- compound: C03264 LIPIDMAPS id: LMFA01050402
('Lipids [LM]', 'Fatty Acyls [FA]', 'Fatty Acids and Conjugates [FA01]', 'Hydroxy fatty acids [FA0105]')

---- compound: HMDB00665 
---- compound: HMDB00746 
---- compound: HMDB01624 
---- compound: HMDB01975 
---- compound: HMDB10718 
---- compound: HMDB12843 
---- compound: LMFA01050011 LIPIDMAPS id: LMFA01050011
('Lipids [LM]', 'Fatty Acyls [FA]

In [24]:
elapsed_time = time.time() - start_time
m, s = divmod(elapsed_time, 60)
print ("Created class annotation columns in " + "%02dm%02ds" % (m, s))

Created class annotation columns in 00m16s


In [25]:
# Second pass: find if present in plants.
df['KNApSAcK'] = df['LipidMaps_2_KEGG'].apply(knapsack_plants)
# Apenas os codigos KEGG dão acesso ao KNApSAcK, como tal a pesquisa no knapsack é feita esta coluna.

53% done
57% done
60% done
63% done
67% done
70% done
73% done
77% done
80% done
83% done
87% done
90% done
93% done
97% done
100% done


In [26]:
# Guarda o novo dataframe para um ficheiro EXCEL (.xlsx) com o mesmo nome do ficheiro .tsv, acrecentado '_raw' ao nome
writer = pd.ExcelWriter(testfile_name[:-4]+'_raw.xlsx')
df.to_excel(writer, header=True, index=False)
writer.save()

elapsed_time = time.time() - start_time
m, s = divmod(elapsed_time, 60)
print ("Finished in " + "%02dm%02ds" % (m, s))

Finished in 00m21s
